# Download Data

In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import rasterio

sys.path.append(os.path.abspath(".."))  

In [2]:
from src.utils import load_config
from src.download_LAI_FAPAR import ProductDownloader

/Users/ivanapallister/WorkSpace/net-primary-production/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
config = load_config()

PRODUCTS = config["products"]
TILES = config["tiles"]
DATES = config["dates"]
SAVE_PATH = config["save_path"]

downloader = ProductDownloader(
    products=PRODUCTS,
    tiles=TILES,
    dates=DATES,
    save_path=SAVE_PATH,
)

downloader.download_all()


Directory already exists: ../data/LAI_FAPAR
Authenticated using refresh token.


KeyboardInterrupt: 

# Compare Data?

In [ ]:
vi_file = "./NPP_inputs/CLMS_products/VI_20220510T104621_S2A_T31UGT-010m_V101_LAI_FIXED_Qgis.tif"
openeo_file = "./FAPAR_T31UFT_2022-05-10.tiff"

In [ ]:
# TODO: add open_data and vi_data
lai_open_decoded = np.where(open_data == 255, np.nan, open_data * 0.1)
vi_data_masked = np.where(vi_data == -32768, np.nan, vi_data)


In [ ]:

with rasterio.open(vi_file) as src_vi, rasterio.open(openeo_file) as src_open:
    vi_data = src_vi.read(1).astype(float)
    open_data = src_open.read(1).astype(float)

    min_rows = min(vi_data.shape[0], open_data.shape[0])
    min_cols = min(vi_data.shape[1], open_data.shape[1])

    vi_data = vi_data[:min_rows, :min_cols]
    open_data = open_data[:min_rows, :min_cols]


diff = lai_open_decoded - vi_data_masked
diff_abs = np.abs(diff)

print(f"Mean abs difference: {np.nanmean(diff_abs):.2f}")
print(f"Max difference: {np.nanmax(diff_abs):.2f}")

# Assuming you've already created:
# - lai_open_decoded
# - vi_data_masked
# - diff = lai_open_decoded - vi_data_masked

plt.figure(figsize=(10, 5))
plt.hist(vi_data_masked[~np.isnan(vi_data_masked)].flatten(), bins=50, alpha=0.5, label='VI (float)')
plt.hist(lai_open_decoded[~np.isnan(lai_open_decoded)].flatten(), bins=50, alpha=0.5, label='OpenEO (scaled)')
plt.title("Value Distribution: LAI (May 10, 2022, T31UFT)")
plt.xlabel("LAI Value")
plt.ylabel("Frequency")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 8))
masked_diff = np.ma.masked_invalid(diff)
plt.imshow(masked_diff, cmap='bwr', vmin=-2, vmax=2)  # Clip to LAI difference ±2
plt.colorbar(label='LAI Difference (OpenEO - VI)')
plt.title("Spatial Difference: LAI (OpenEO - VI)")
plt.axis('off')
plt.tight_layout()
plt.show()

#TODO: I guess he is comparing the two datasources but I am a bit confused about the context of this code.